#### 사용자 정의 데이터셋
- pytorch에서 딥러닝 시 대량의 데이터 사용에 따른 부하(H/w, s/w) 및 많은 시간 소요에 대한 해결책으로 제시
- 대량 데이터셋 전용 처리 모듈 제공
- DataSet과 DataLoader
    - DataSet=> 데이터 전처리, 턴서화 등의 작업 진행
    - DataLoader=> Dataset 인스턴스를 사용해서 배치크기 만큼 데이터를 추출함

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [18]:
DATAFILE='../data/iris.csv'
irisDF=pd.read_csv(DATAFILE)


In [19]:
encoder=LabelEncoder()
encoder.fit(irisDF['variety'])
irisDF['variety']=encoder.transform(irisDF['variety'])

[2] 사용자 정의 데이터셋 클래스 생성 <hr>

In [20]:
# 클래스 목적: 학습용 데이터셋 텐서화 및 전처리
# 클래스 이름: CustomDataset
# 부모 클래스: torch.utils.data.Dataset
# 매개변수: featureDF, TargetDF


In [21]:
class CustomDataset(Dataset):
    def __init__(self, featureDF, targetDF):
        super().__init__()
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]
        

    # 데이터의 개수 반환 메서드
    def __len__(self):
        return self.n_rows
    
    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor반환!
    def __getitem__(self, idx):
        featureTS= torch.FloatTensor(self.featureDF.iloc[idx].values)
        targetTS= torch.FloatTensor(self.targetDF.iloc[idx].values)
        return featureTS, targetTS

In [22]:
def convertDataFrame(data_path, exit_header):
    ext=data_path.split('.')[-1]
    if ext == 'csv':
        return pd.read_csv(data_path, header=0 if exit_header else None)
    elif ext == 'json':
        return pd.read_json(data_path,  header=0 if exit_header else None)
    elif ext in ['xlsx', 'xls']:
        return pd.read_excel(data_path,  header=0 if exit_header else None)
    else:
        return pd.read_table(data_path, header=0 if exit_header else None)

In [23]:
# 파일기반 데이터셋일 경우
# 클래스이름: FileDataset
# 매개변수: data_path, 
# 부모 클래스: utils.data.Dataset
# ___________________________________________________________________________________________
class FileDataset(Dataset):
    def __init__(self, data_path):
        super().__init__()
        #확장자 추출
        dataDF=convertDataFrame(data_path)
        self.featuresDF=dataDF.colums[:-1]
        self.targetDF= dataDF[dataDF.columns[-1]]
        
        self.n_features=self.featuresDF.shape[0]
        self.n_rows= self.featuresDF.shape[1]


    # 데이터의 개수 반환 메서드
    def __len__(self):
        return self.n_rows
    
    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor반환!
    def __getitem__(self, idx):
        featureTS= torch.FloatTensor(self.featureDF.iloc[idx].values)
        targetTS= torch.FloatTensor(self.targetDF.iloc[idx].values)
        return featureTS, targetTS

[3] 데이터셋 인스턴스 생성 <hr>

In [24]:
featureDF, targetDF= irisDF[irisDF.columns[:-1]], irisDF[[irisDF.columns[-1]]]
print(targetDF.shape, featureDF.shape)
irisDF= CustomDataset(featureDF, targetDF)

(150, 1) (150, 4)


In [25]:

irisDF.n_features,irisDF.n_rows,

(4, 150)

[4] 데이터로더 인스턴스 생성


In [26]:
# 필요한 것: Dataset 인스턴스, batch_size=1(기본)
irisDL=DataLoader(irisDF, batch_size=32)    #iterable

In [27]:
targetTS

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])

In [28]:
for dataTS, targetTS in irisDL:
    print(dataTS.shape, targetTS.shape)
    break

torch.Size([32, 4]) torch.Size([32, 1])
